<a href="https://colab.research.google.com/github/bsenursahin/UYB455YAPAYZEKA/blob/main/BERTFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Kütüphaneleri kuruyoruz.

!pip install transformers accelerate datasets scikit-learn openpyxl -q

In [ ]:
#BERT MODELİ

import os
import gc
import warnings
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from datasets import Dataset

# Uyarıları kapat
warnings.filterwarnings('ignore')
os.environ["WANDB_DISABLED"] = "true"


torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

class Config:
    """Training Configuration - A100 OPTIMIZED & EXCEL FIX"""

    FILE_PATH = "/content/temiz_veri_seti.xlsx" # Excel dosyan

    MODEL_NAME = "dbmdz/bert-base-turkish-cased"
    OUTPUT_DIR = "./turkish_sentiment_model"

    TEXT_COLUMN = "Yorum"
    LABEL_COLUMN = "Duygu"

    # --- A100 PERFORMANS AYARLARI ---
    MAX_LENGTH = 128
    TRAIN_BATCH_SIZE = 256
    EVAL_BATCH_SIZE = 512
    NUM_EPOCHS = 3
    LEARNING_RATE = 2e-5
    WARMUP_RATIO = 0.1
    WEIGHT_DECAY = 0.01
    TEST_SIZE = 0.10
    VAL_SIZE = 0.10
    RANDOM_STATE = 42
    GRADIENT_ACCUMULATION_STEPS = 1
    FP16 = True
    GRADIENT_CHECKPOINTING = False
    MAX_SAMPLES = None
    NUM_WORKERS = 8


def load_and_prepare_data(config):
    """Load and preprocess dataset (Excel & CSV Support)"""

    print("Data Loading")
    print("-" * 70)

    try:
        print(f"\nLoading: {config.FILE_PATH}")

        # --- DÜZELTME BURADA: Dosya uzantısına göre okuma ---
        if config.FILE_PATH.endswith('.xlsx') or config.FILE_PATH.endswith('.xls'):
            print("Excel formatı algılandı, pd.read_excel kullanılıyor...")
            df = pd.read_excel(config.FILE_PATH)
        else:
            # CSV okuma denemeleri (eskisi gibi)
            print("CSV formatı varsayılıyor, pd.read_csv kullanılıyor...")
            try:
                df = pd.read_csv(config.FILE_PATH, sep=';', on_bad_lines='skip', encoding='utf-8')
            except:
                try:
                    df = pd.read_csv(config.FILE_PATH, sep=',', on_bad_lines='skip', encoding='utf-8')
                except:
                    df = pd.read_csv(config.FILE_PATH, sep=';', on_bad_lines='skip', encoding='windows-1254')

        # Sütun isim temizliği
        df.columns = df.columns.str.replace('ï»¿', '', regex=False).str.strip()

        # Boşları temizle
        df = df.drop_duplicates(subset=[config.TEXT_COLUMN])
        df = df.dropna(subset=[config.TEXT_COLUMN, config.LABEL_COLUMN])

        print(f"Total Rows: {len(df):,}")

        # Örneklem (opsiyonel)
        if config.MAX_SAMPLES and len(df) > config.MAX_SAMPLES:
            df = df.sample(n=config.MAX_SAMPLES, random_state=config.RANDOM_STATE)

        print("\nMapping labels...")
        # Eğer etiketler zaten 0,1,2 ise dokunmaz, string ise çevirir
        label_map = {
            'Olumsuz': 0, 'Negative': 0, 'negative': 0,
            'Olumlu': 1, 'Positive': 1, 'positive': 1,
            'Nötr': 2, 'Neutral': 2, 'neutral': 2, 'Notr': 2
        }

        # Etiket sütunu string ise map'le, değilse olduğu gibi bırak (integer kontrolü)
        if df[config.LABEL_COLUMN].dtype == 'object':
             df['label'] = df[config.LABEL_COLUMN].astype(str).str.strip().map(label_map)
        else:
             df['label'] = df[config.LABEL_COLUMN]

        # Temizlik
        df = df.dropna(subset=['label'])
        df['label'] = df['label'].astype(int)

        print("Class Distribution:")
        print(df['label'].value_counts().sort_index())

        return df

    except FileNotFoundError:
        raise FileNotFoundError(f"File not found: {config.FILE_PATH}")
    except Exception as e:
        raise Exception(f"Error loading data: {str(e)}")


def split_data(df, config):
    print("\nData Splitting")
    print("-" * 70)
    train_val, test = train_test_split(
        df, test_size=config.TEST_SIZE,
        random_state=config.RANDOM_STATE, stratify=df['label']
    )
    val_size_adjusted = config.VAL_SIZE / (1 - config.TEST_SIZE)
    train, val = train_test_split(
        train_val, test_size=val_size_adjusted,
        random_state=config.RANDOM_STATE, stratify=train_val['label']
    )
    print(f"Train: {len(train):,}")
    print(f"Val:   {len(val):,}")
    print(f"Test:  {len(test):,}")
    return train.reset_index(drop=True), val.reset_index(drop=True), test.reset_index(drop=True)


def create_datasets(train_df, val_df, test_df, tokenizer, config):
    print("\nTokenization")
    print("-" * 70)
    def tokenize_function(examples):
        return tokenizer(
            examples[config.TEXT_COLUMN],
            padding="max_length",
            truncation=True,
            max_length=config.MAX_LENGTH
        )
    train_dataset = Dataset.from_pandas(train_df[[config.TEXT_COLUMN, 'label']])
    val_dataset = Dataset.from_pandas(val_df[[config.TEXT_COLUMN, 'label']])
    test_dataset = Dataset.from_pandas(test_df[[config.TEXT_COLUMN, 'label']])

    train_dataset = train_dataset.map(tokenize_function, batched=True, batch_size=5000)
    val_dataset = val_dataset.map(tokenize_function, batched=True, batch_size=5000)
    test_dataset = test_dataset.map(tokenize_function, batched=True, batch_size=5000)

    train_dataset = train_dataset.remove_columns([config.TEXT_COLUMN])
    val_dataset = val_dataset.remove_columns([config.TEXT_COLUMN])
    test_dataset = test_dataset.remove_columns([config.TEXT_COLUMN])

    train_dataset.set_format("torch")
    val_dataset.set_format("torch")
    test_dataset.set_format("torch")
    gc.collect()
    torch.cuda.empty_cache()
    return train_dataset, val_dataset, test_dataset


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1_macro": f1_score(labels, predictions, average='macro'),
        "precision_macro": precision_score(labels, predictions, average='macro'),
        "recall_macro": recall_score(labels, predictions, average='macro')
    }


def train_model(config):
    print("Turkish BERT Sentiment Analysis Training (A100 Optimized)")
    print("-" * 70)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")

    df = load_and_prepare_data(config)
    train_df, val_df, test_df = split_data(df, config)
    del df
    gc.collect()

    print("\nModel Loading")
    tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)
    model = AutoModelForSequenceClassification.from_pretrained(
        config.MODEL_NAME, num_labels=3,
        id2label={0: "Negative", 1: "Positive", 2: "Neutral"},
        label2id={"Negative": 0, "Positive": 1, "Neutral": 2}
    )

    if config.GRADIENT_CHECKPOINTING:
        model.gradient_checkpointing_enable()

    train_dataset, val_dataset, test_dataset = create_datasets(
        train_df, val_df, test_df, tokenizer, config
    )
    del train_df, val_df, test_df
    gc.collect()

    training_args = TrainingArguments(
        output_dir=config.OUTPUT_DIR,
        num_train_epochs=config.NUM_EPOCHS,
        per_device_train_batch_size=config.TRAIN_BATCH_SIZE,
        per_device_eval_batch_size=config.EVAL_BATCH_SIZE,
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
        learning_rate=config.LEARNING_RATE,
        warmup_ratio=config.WARMUP_RATIO,
        weight_decay=config.WEIGHT_DECAY,
        logging_dir=f"{config.OUTPUT_DIR}/logs",
        logging_steps=100,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        greater_is_better=True,
        fp16=config.FP16,
        dataloader_num_workers=config.NUM_WORKERS,
        save_total_limit=1,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
    )

    print("\nStarting Training...")
    trainer.train()

    print("\nEvaluation Results:")
    print(trainer.evaluate())
    print("\nTest Results:")
    print(trainer.evaluate(test_dataset))

    print("\nSaving Model...")
    trainer.save_model(config.OUTPUT_DIR)
    tokenizer.save_pretrained(config.OUTPUT_DIR)
    print("Done.")
    return trainer


if __name__ == "__main__":
    config = Config()
    trainer = train_model(config)

Turkish BERT Sentiment Analysis Training (A100 Optimized)
----------------------------------------------------------------------
Device: cuda
GPU: NVIDIA A100-SXM4-40GB
Data Loading
----------------------------------------------------------------------

Loading: /content/temiz_veri_seti.xlsx
Excel formatı algılandı, pd.read_excel kullanılıyor...
Total Rows: 488,872

Mapping labels...
Class Distribution:
label
0     56275
1    261681
2    170916
Name: count, dtype: int64

Data Splitting
----------------------------------------------------------------------
Train: 391,096
Val:   48,888
Test:  48,888

Model Loading


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

...

Map:   0%|          | 0/48888 [00:00<?, ? examples/s]


Starting Training...


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Macro,Recall Macro
1,0.081400,0.080361,0.972570,0.951801,0.957258,0.946686
2,0.067000,0.076141,0.974656,0.955309,0.961116,0.949886
3,0.045600,0.080629,0.974963,0.955591,0.963605,0.948276



Evaluation Results:


{'eval_loss': 0.08062899112701416, 'eval_accuracy': 0.9749631811487481, 'eval_f1_macro': 0.9555906910484887, 'eval_precision_macro': 0.9636053968677939, 'eval_recall_macro': 0.9482764580902473, 'eval_runtime': 9.652, 'eval_samples_per_second': 5065.075, 'eval_steps_per_second': 9.946, 'epoch': 3.0}

Test Results:
{'eval_loss': 0.0804213359951973, 'eval_accuracy': 0.9743086237931599, 'eval_f1_macro': 0.9544951448856073, 'eval_precision_macro': 0.9624020504282188, 'eval_recall_macro': 0.9472715588827336, 'eval_runtime': 9.6564, 'eval_samples_per_second': 5062.77, 'eval_steps_per_second': 9.942, 'epoch': 3.0}

Saving Model...
Done.


In [ ]:
# 1. Google Drive'ı Bağla (İzin isteyecek, onayla)
from google.colab import drive
drive.mount('/content/drive')

# 2. Modeli Drive'a 'tez_model_v1' adıyla kopyala
!cp -r ./turkish_sentiment_model /content/drive/MyDrive/tez_model_v1

print("✅ Model Google Drive'a başarıyla yedeklendi: /content/drive/MyDrive/tez_model_v1")

Mounted at /content/drive
✅ Model Google Drive'a başarıyla yedeklendi: /content/drive/MyDrive/tez_model_v1


In [ ]:
#Kaggle üzerindeki veri setini düzenleyelim. (Dataset sütunu kaldırılacak, label sayısallaştırılacak.)

!pip install datasets pandas openpyxl -q

from datasets import load_dataset
import pandas as pd

DATASET_NAME = "winvoker/turkish-sentiment-analysis-dataset"

print(f"{DATASET_NAME} indiriliyor...")
dataset = load_dataset(DATASET_NAME)

dfs = []
for split in dataset.keys():
    dfs.append(pd.DataFrame(dataset[split]))
df = pd.concat(dfs, ignore_index=True)

print(f"İlk Sütunlar: {df.columns.tolist()}")

if 'dataset' in df.columns:
    df = df.drop(columns=['dataset'])
    print("'dataset' sütunu silindi.")

df = df.rename(columns={
    "text": "Yorum",
    "label": "Duygu"
})

label_map = {
    'Negative': 0, 'negative': 0, 'Olumsuz': 0,
    'Positive': 1, 'positive': 1, 'Olumlu': 1,
    'Neutral': 2,  'neutral': 2,  'Nötr': 2, 'Notr': 2
}

if df['Duygu'].dtype == 'object':
    df['Duygu'] = df['Duygu'].map(label_map)
    print("Etiketler (positive/negative) sayısal formata (0/1/2) çevrildi.")

df = df.dropna(subset=['Duygu', 'Yorum'])
df['Duygu'] = df['Duygu'].astype(int)

print("-" * 30)
print("Son Durum:")
print(df.head())
print("\nEtiket Dağılımı:\n", df['Duygu'].value_counts())

dosya_adi = "temiz_veri_seti.xlsx"
df.to_excel(dosya_adi, index=False)

print(f"\nTAMAMLANDI! '{dosya_adi}' dosyasını sol menüden indirip eğitimde kullanabilirsin.")

winvoker/turkish-sentiment-analysis-dataset indiriliyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/76.1M [00:00<?, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/440679 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/48965 [00:00<?, ? examples/s]

İlk Sütunlar: ['text', 'label', 'dataset']
'dataset' sütunu silindi.
Etiketler (positive/negative) sayısal formata (0/1/2) çevrildi.
------------------------------
Son Durum:
                                               Yorum  Duygu
0  ürünü hepsiburadadan alalı 3 hafta oldu. orjin...      1
1  ürünlerden çok memnunum, kesinlikle herkese ta...      1
2      hızlı kargo, temiz alışveriş.teşekkür ederim.      1
3               Çünkü aranan tapınak bu bölgededir .      2
4  bu telefonu başlıca alma nedenlerim ise elimde...      1

Etiket Dağılımı:
 Duygu
1    262166
2    170917
0     56561
Name: count, dtype: int64

TAMAMLANDI! 'temiz_veri_seti.xlsx' dosyasını sol menüden indirip eğitimde kullanabilirsin.


In [ ]:
import pandas as pd

sutun_isimleri = ["ID", "Category", "Review", "Sentiment", "Split", "Label"]

try:
    df = pd.read_csv('/content/HUMIRSentimentDatasets.csv',
                     header=None,          # İlk satırı başlık yapma, veri olarak al
                     names=sutun_isimleri, # Sütun isimlerini biz veriyoruz
                     sep=None,             # Ayırıcıyı (virgül mü tab mı) otomatik algıla
                     engine='python',      # Python motoru hatalara karşı daha esnektir
                     on_bad_lines='skip')  # Hatalı olan o 29. satırı atla ve devam et

    print("✅ Veri başarıyla yüklendi!")
    print(df.head())
    print(f"\nToplam Okunan Satır Sayısı: {len(df)}")

except Exception as e:
    print(f"❌ Hata: {e}")

✅ Veri başarıyla yüklendi!
       ID      Category                                             Review  \
0  ﻿80101  Hotel Review  asla gidilmeyecek bir otel hasta oldukotel tam...   
1   80102  Hotel Review  berbatbalayı için tercih etmiş olduğumuz bir o...   
2   80103  Hotel Review  rexaletodalar felaket hamamböcekleri toz lobby...   
3   80104  Hotel Review  berbat personeliyim otelin mutlaka okuyunmerha...   
4   80105  Hotel Review  sıkıcı açözel sektör çalışanı olduğum ve önümü...   

  Sentiment  Split  Label  
0  Negative  train      1  
1  Negative  train      1  
2  Negative  train      1  
3  Negative  train      1  
4  Negative  train      1  

Toplam Okunan Satır Sayısı: 65000


In [ ]:
print("--- Veri Seti Bölümleri (Split) ---")
print(df['Split'].value_counts())

print("\n--- Duygu Dağılımı (Sentiment) ---")
print(df['Sentiment'].value_counts())

print("\n--- Etiket Dağılımı (Label) ---")
print(df['Label'].value_counts())

# Eksik veri kontrolü (özellikle Review sütununda)
print(f"\nBoş Yorum Sayısı: {df['Review'].isnull().sum()}")

--- Veri Seti Bölümleri (Split) ---
Split
train    32500
test     32500
Name: count, dtype: int64

--- Duygu Dağılımı (Sentiment) ---
Sentiment
Negative    32500
Positive    32500
Name: count, dtype: int64

--- Etiket Dağılımı (Label) ---
Label
1    22280
2    10680
3    10680
4    10680
5    10680
Name: count, dtype: int64

Boş Yorum Sayısı: 0


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader

# --- 1. DOSYAYI OKUMA (En baştaki adım) ---
print("1. Dosya okunuyor...")
sutun_isimleri = ["ID", "Category", "Review", "Sentiment", "Split", "Label"]
try:
    df = pd.read_csv('/content/HUMIRSentimentDatasets.csv',
                     header=None,
                     names=sutun_isimleri,
                     sep=None,
                     engine='python',
                     on_bad_lines='skip')
    print("Dosya başarıyla yüklendi.")
except Exception as e:
    print(f" Dosya okunurken hata: {e}")

# --- 2. VERİYİ HAZIRLAMA ---
print("2. Veriler düzenleniyor...")
# Etiketleri Sayıya Çevirme
label_map = {'Negative': 0, 'Positive': 1}
df['target'] = df['Sentiment'].map(label_map)

# Train ve Test olarak ayırma (Split sütununu kullanarak)
train_raw = df[df['Split'] == 'train'].copy()
test_df = df[df['Split'] == 'test'].copy()

# Validation oluşturma (Train içinden %10 ayırıyoruz)
train_df, val_df = train_test_split(train_raw, test_size=0.1, random_state=42, stratify=train_raw['target'])

# Tokenizer'ı yükleme
MODEL_NAME = "dbmdz/bert-base-turkish-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(" Tokenizer hazır.")

# --- 3. DATASET CLASS ve LOADER ---
print("3. Veri Yükleyiciler oluşturuluyor...")

class ReviewDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        review = str(self.df.iloc[index]['Review'])
        target = self.df.iloc[index]['target']

        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

# Ayarlar
MAX_LEN = 160
BATCH_SIZE = 16

# Datasetleri Oluştur
train_dataset = ReviewDataset(train_df, tokenizer, MAX_LEN)
val_dataset = ReviewDataset(val_df, tokenizer, MAX_LEN)
test_dataset = ReviewDataset(test_df, tokenizer, MAX_LEN)

# Loaderları Oluştur
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print("✅ HER ŞEY HAZIR! Eğitime geçebiliriz.")

1. Dosya okunuyor...
✅ Dosya başarıyla yüklendi.
2. Veriler düzenleniyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

✅ Tokenizer hazır.
3. Veri Yükleyiciler oluşturuluyor...
✅ HER ŞEY HAZIR! Eğitime geçebiliriz.


In [ ]:
"""
BERT/RoBERTa Fine-Tuning Script - A100 Optimized
 3 sınıflı modeli 2 sınıfa uyarlar
 Yeni model olarak kaydeder (tez_model_v2)
 A100 GPU için optimize edilmiş
 Hata kontrolü %100
"""

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from google.colab import drive
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW  #  DÜZELTME: transformers'tan değil torch'tan import
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    recall_score,
    precision_score,
    classification_report,
    confusion_matrix
)
import warnings
import json
import os
from datetime import datetime
from tqdm.auto import tqdm

warnings.filterwarnings('ignore')


CONFIG = {
    'ORIGINAL_MODEL_NAME': 'tez_model_v1',    # Drive'daki mevcut model
    'NEW_MODEL_NAME': 'tez_model_v2',         # Yeni model adı

    'NUM_LABELS': 2,                          # Yeni veri setindeki sınıf sayısı
    'LABEL_NAMES': ['Negative', 'Positive'],  # Sınıf isimleri

    'EPOCHS': 3,
    'BATCH_SIZE': None,  # None = otomatik tespit (val_loader'dan alır)
    'LEARNING_RATE': 2e-5,
    'WARMUP_RATIO': 0.1,
    'MAX_GRAD_NORM': 1.0,
    'WEIGHT_DECAY': 0.01,

    'AUTO_SAVE': True,                        # True = otomatik kaydet
    'SAVE_BEST_ONLY': False,                  # True = sadece en iyi modeli kaydet
}

print("="*70)
print(" BERT/RoBERTa FINE-TUNING BAŞLIYOR")
print("="*70)
print(f" Kaynak model    : {CONFIG['ORIGINAL_MODEL_NAME']}")
print(f" Hedef model     : {CONFIG['NEW_MODEL_NAME']}")
print(f" Sınıf sayısı    : 3 → {CONFIG['NUM_LABELS']}")
print("="*70)

# =============================================================================
# 0. GÜVENLİK KONTROLLERI
# =============================================================================

# Gerekli değişkenleri kontrol et
required_vars = {
    'train_loader': 'Eğitim veri yükleyici',
    'val_loader': 'Validasyon veri yükleyici',
    'train_df': 'Eğitim dataframe'
}

missing = []
for var_name, description in required_vars.items():
    if var_name not in dir():
        missing.append(f" {var_name} ({description})")
    else:
        print(f"  ✅ {var_name} bulundu")

if missing:
    print("\n HATA: Aşağıdaki değişkenler bulunamadı:")
    print("\n".join(missing))
    print("\nÇÖZÜM: Önce veri hazırlama kodunu çalıştırın!")
    raise SystemExit(1)

# Batch size'ı otomatik tespit et
if CONFIG['BATCH_SIZE'] is None:
    try:
        CONFIG['BATCH_SIZE'] = train_loader.batch_size
        print(f"  Batch size otomatik tespit edildi: {CONFIG['BATCH_SIZE']}")
    except:
        CONFIG['BATCH_SIZE'] = 16
        print(f"  Batch size tespit edilemedi, varsayılan: {CONFIG['BATCH_SIZE']}")

print("\n Tüm kontroller başarılı!")


print("\n Google Drive bağlanıyor...")
try:
    drive.mount('/content/drive', force_remount=False)
    print("Drive bağlandı")
except Exception as e:
    print(f" Drive bağlantı uyarısı: {e}")
    print("   (Zaten bağlıysa sorun yok)")

print("\n GPU kontrol ediliyor...")
if not torch.cuda.is_available():
    print(" HATA: GPU bulunamadı!")
    print(" ÇÖZÜM: Runtime > Change runtime type > T4/A100 GPU seçin")
    raise SystemExit(1)

device = torch.device("cuda")
print(f"GPU aktif: {torch.cuda.get_device_name(0)}")
print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# A100 için özel optimizasyonlar
if "A100" in torch.cuda.get_device_name(0):
    print("A100 tespit edildi - TF32 optimizasyonları aktif!")
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cudnn.benchmark = True

original_path = f"/content/drive/MyDrive/{CONFIG['ORIGINAL_MODEL_NAME']}"
print(f"\n Model yükleniyor: {original_path}")

# Modelin var olduğunu kontrol et
if not os.path.exists(original_path):
    print(f" HATA: Model klasörü bulunamadı!")
    print(f"   Aranan konum: {original_path}")
    print("\n ÇÖZÜM: Drive'ınızda şu klasör var mı kontrol edin:")
    print(f"   MyDrive/{CONFIG['ORIGINAL_MODEL_NAME']}")
    raise SystemExit(1)

try:
    # Tokenizer'ı yükle
    print("  → Tokenizer yükleniyor...")
    tokenizer = AutoTokenizer.from_pretrained(original_path)

    # Modeli yükle ve 2 sınıfa uyarla
    print("  → Model yükleniyor ve 2 sınıfa uyarlanıyor...")
    print("     (3 sınıflı classifier katmanı → 2 sınıflı olarak yeniden oluşturuluyor)")

    model = AutoModelForSequenceClassification.from_pretrained(
        original_path,
        num_labels=CONFIG['NUM_LABELS'],
        ignore_mismatched_sizes=True  # Bu satır 3→2 uyarlamasını otomatik yapar
    )

    model = model.to(device)

    print("\n Model başarıyla yüklendi ve uyarlandı!")
    print(f"   Model tipi: {model.config.model_type.upper()}")
    print(f"   Sınıf sayısı: {model.num_labels}")
    print(f"   Toplam parametre: {sum(p.numel() for p in model.parameters()):,}")
    print(f"   Eğitilebilir parametre: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

except Exception as e:
    print(f"\n Model yükleme hatası!")
    print(f"   Hata: {e}")
    raise SystemExit(1)


def get_predictions(model, data_loader, device):
    """Model tahminlerini ve gerçek değerleri döndürür"""
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Tahmin yapılıyor", leave=False):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["targets"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)

            predictions.extend(preds.cpu().tolist())
            true_labels.extend(labels.cpu().tolist())

    return true_labels, predictions


def calculate_metrics(y_true, y_pred):
    """Metrikleri hesapla"""
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0)
    }


def train_epoch(model, data_loader, loss_fn, optimizer, scheduler, device):
    """Bir epoch eğitim"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    progress_bar = tqdm(data_loader, desc="Eğitim", leave=False)

    for batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["targets"].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=CONFIG['MAX_GRAD_NORM'])
        optimizer.step()
        scheduler.step()

        # Metrikleri hesapla
        _, preds = torch.max(outputs.logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        total_loss += loss.item()

        # Progress bar güncelle
        progress_bar.set_postfix({
            'loss': f"{loss.item():.4f}",
            'acc': f"{correct/total:.4f}"
        })

    return total_loss / len(data_loader), correct / total


def eval_epoch(model, data_loader, loss_fn, device):
    """Validasyon epoch"""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Validasyon", leave=False):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["targets"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)

            _, preds = torch.max(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

    return total_loss / len(data_loader), correct / total

print("\n" + "="*70)
print(" BASELINE PERFORMANS (Eğitim Öncesi)")
print("="*70)
print(" Model performansı ölçülüyor...")

y_true, y_pred_baseline = get_predictions(model, val_loader, device)
baseline_metrics = calculate_metrics(y_true, y_pred_baseline)

print("\n Sonuçlar:")
for metric, value in baseline_metrics.items():
    print(f"  {metric.upper():12s}: {value:.4f}")


print("\n" + "="*70)
print("🎓 EĞİTİM HAZIRLIĞI")
print("="*70)

# Optimizer
optimizer = AdamW(
    model.parameters(),
    lr=CONFIG['LEARNING_RATE'],
    eps=1e-8,
    weight_decay=CONFIG['WEIGHT_DECAY']
)

# Scheduler
total_steps = len(train_loader) * CONFIG['EPOCHS']
warmup_steps = int(CONFIG['WARMUP_RATIO'] * total_steps)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# Loss function
loss_fn = nn.CrossEntropyLoss().to(device)

print(f" Optimizer hazır (AdamW, lr={CONFIG['LEARNING_RATE']})")
print(f" Scheduler hazır (warmup={warmup_steps} steps)")
print(f" Loss function hazır (CrossEntropyLoss)")
print(f"\n Eğitim detayları:")
print(f"  • Epoch sayısı: {CONFIG['EPOCHS']}")
print(f"  • Batch size: {CONFIG['BATCH_SIZE']}")
print(f"  • Toplam step: {total_steps}")
print(f"  • Warmup step: {warmup_steps}")


print("\n" + "="*70)
print(" EĞİTİM BAŞLIYOR")
print("="*70)

history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

best_val_acc = 0.0

for epoch in range(CONFIG['EPOCHS']):
    print(f"\n📅 EPOCH {epoch + 1}/{CONFIG['EPOCHS']}")
    print("-" * 70)

    # Training
    train_loss, train_acc = train_epoch(
        model, train_loader, loss_fn, optimizer, scheduler, device
    )

    # Validation
    val_loss, val_acc = eval_epoch(model, val_loader, loss_fn, device)

    # Kayıt
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)

    # Best model tracking
    is_best = val_acc > best_val_acc
    if is_best:
        best_val_acc = val_acc

    # Sonuçları yazdır
    print(f"\n{'':20s} Loss      Accuracy")
    print(f"  Train:         {train_loss:8.4f}  {train_acc:8.4f}")
    print(f"  Validation:    {val_loss:8.4f}  {val_acc:8.4f} {'🌟 BEST!' if is_best else ''}")

print("\n✅ Eğitim tamamlandı!")



print("\n" + "="*70)
print(" FİNAL PERFORMANS (Eğitim Sonrası)")
print("="*70)
print(" Model performansı ölçülüyor...")

y_true, y_pred_final = get_predictions(model, val_loader, device)
final_metrics = calculate_metrics(y_true, y_pred_final)

print("\n📈 Sonuçlar:")
for metric, value in final_metrics.items():
    print(f"  {metric.upper():12s}: {value:.4f}")



print("\n" + "="*70)
print(" BASELINE vs FİNAL KARŞILAŞTIRMA")
print("="*70)

comparison = pd.DataFrame({
    'Metrik': ['Accuracy', 'F1 Score', 'Recall', 'Precision'],
    'Baseline': [baseline_metrics[k] for k in ['accuracy', 'f1', 'recall', 'precision']],
    'Final': [final_metrics[k] for k in ['accuracy', 'f1', 'recall', 'precision']]
})

comparison['Fark'] = comparison['Final'] - comparison['Baseline']
comparison['Değişim (%)'] = ((comparison['Final'] / comparison['Baseline']) - 1) * 100

print(comparison.to_string(index=False, formatters={
    'Baseline': '{:.4f}'.format,
    'Final': '{:.4f}'.format,
    'Fark': '{:+.4f}'.format,
    'Değişim (%)': '{:+.2f}%'.format
}))

# Detaylı rapor
print("\n" + "="*70)
print("DETAYLI SINIFLANDIRMA RAPORU")
print("="*70)
print(classification_report(y_true, y_pred_final,
                          target_names=CONFIG['LABEL_NAMES'],
                          digits=4))

# Confusion Matrix
print(" Confusion Matrix:")
cm = confusion_matrix(y_true, y_pred_final)
print(cm)



save_model = CONFIG['AUTO_SAVE']

if not save_model:
    response = input("\n💾 Modeli kaydetmek ister misiniz? (e/h): ").lower()
    save_model = (response == 'e')

if save_model:
    print("\n" + "="*70)
    print(" MODEL KAYDEDİLİYOR")
    print("="*70)

    save_path = f"/content/drive/MyDrive/{CONFIG['NEW_MODEL_NAME']}"

    # Klasör oluştur
    os.makedirs(save_path, exist_ok=True)

    # Model ve tokenizer'ı kaydet
    print(f" Kaydediliyor: {save_path}")
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    # Config dosyası kaydet
    config_save = CONFIG.copy()
    config_save['training_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    config_save['baseline_metrics'] = baseline_metrics
    config_save['final_metrics'] = final_metrics
    config_save['training_history'] = history
    config_save['gpu_used'] = torch.cuda.get_device_name(0)

    with open(f"{save_path}/training_info.json", 'w', encoding='utf-8') as f:
        json.dump(config_save, f, ensure_ascii=False, indent=2)

    print(" Model kaydedildi!")
    print(f" Eğitim bilgileri kaydedildi: training_info.json")

    print("\n Kaydedilen dosyalar:")
    print(f"  • Model: {save_path}/pytorch_model.bin")
    print(f"  • Config: {save_path}/config.json")
    print(f"  • Tokenizer: {save_path}/tokenizer_config.json")
    print(f"  • Training Info: {save_path}/training_info.json")
else:
    print("\n⏭ Model kaydedilmedi")


print("\n" + "="*70)
print(" İŞLEM TAMAMLANDI!")
print("="*70)
print(f" Kaynak model: {CONFIG['ORIGINAL_MODEL_NAME']}")
print(f" Yeni model: {CONFIG['NEW_MODEL_NAME']}")
print(f" Sınıf sayısı: 3 → {CONFIG['NUM_LABELS']}")
print(f" Final Accuracy: {final_metrics['accuracy']:.4f}")
print(f" Gelişme: {comparison[comparison['Metrik']=='Accuracy']['Değişim (%)'].values[0]:+.2f}%")

if save_model:
    print(f"\n Model şurada: MyDrive/{CONFIG['NEW_MODEL_NAME']}")
    print(" Bir sonraki eğitim için: 'tez_model_v2' → 'tez_model_v3'")

print("\n Tebrikler! Her şey başarılı!")
print("="*70)

🚀 BERT/RoBERTa FINE-TUNING BAŞLIYOR
📥 Kaynak model    : tez_model_v1
📤 Hedef model     : tez_model_v2
🎯 Sınıf sayısı    : 3 → 2

🔍 Sistem kontrolleri yapılıyor...
  ✅ train_loader bulundu
  ✅ val_loader bulundu
  ✅ train_df bulundu
  ✅ Batch size otomatik tespit edildi: 16

✅ Tüm kontroller başarılı!

📡 Google Drive bağlanıyor...
Mounted at /content/drive
✅ Drive bağlandı

🖥️ GPU kontrol ediliyor...
✅ GPU aktif: NVIDIA A100-SXM4-40GB
   VRAM: 42.5 GB
🚀 A100 tespit edildi - TF32 optimizasyonları aktif!

📥 Model yükleniyor: /content/drive/MyDrive/tez_model_v1
  → Tokenizer yükleniyor...
  → Model yükleniyor ve 2 sınıfa uyarlanıyor...
     (3 sınıflı classifier katmanı → 2 sınıflı olarak yeniden oluşturuluyor)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/tez_model_v1 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



✅ Model başarıyla yüklendi ve uyarlandı!
   Model tipi: BERT
   Sınıf sayısı: 2
   Toplam parametre: 110,618,882
   Eğitilebilir parametre: 110,618,882

📊 BASELINE PERFORMANS (Eğitim Öncesi)
⏳ Model performansı ölçülüyor...


Tahmin yapılıyor:   0%|          | 0/204 [00:00<?, ?it/s]


📈 Sonuçlar:
  ACCURACY    : 0.4868
  F1          : 0.4121
  RECALL      : 0.4868
  PRECISION   : 0.4731

🎓 EĞİTİM HAZIRLIĞI
✅ Optimizer hazır (AdamW, lr=2e-05)
✅ Scheduler hazır (warmup=548 steps)
✅ Loss function hazır (CrossEntropyLoss)

📊 Eğitim detayları:
  • Epoch sayısı: 3
  • Batch size: 16
  • Toplam step: 5487
  • Warmup step: 548

🚀 EĞİTİM BAŞLIYOR

📅 EPOCH 1/3
----------------------------------------------------------------------


...

Eğitim:   0%|          | 0/1829 [00:00<?, ?it/s]

Validasyon:   0%|          | 0/204 [00:00<?, ?it/s]


                     Loss      Accuracy
  Train:           0.1570    0.9524
  Validation:      0.4005    0.8960 

✅ Eğitim tamamlandı!

📊 FİNAL PERFORMANS (Eğitim Sonrası)
⏳ Model performansı ölçülüyor...


Tahmin yapılıyor:   0%|          | 0/204 [00:00<?, ?it/s]


📈 Sonuçlar:
  ACCURACY    : 0.8960
  F1          : 0.8960
  RECALL      : 0.8960
  PRECISION   : 0.8962

📊 BASELINE vs FİNAL KARŞILAŞTIRMA
   Metrik Baseline  Final    Fark Değişim (%)
 Accuracy   0.4868 0.8960 +0.4092     +84.07%
 F1 Score   0.4121 0.8960 +0.4839    +117.40%
   Recall   0.4868 0.8960 +0.4092     +84.07%
Precision   0.4731 0.8962 +0.4230     +89.41%

📋 DETAYLI SINIFLANDIRMA RAPORU
              precision    recall  f1-score   support

    Negative     0.9040    0.8862    0.8950      1625
    Positive     0.8884    0.9058    0.8970      1625

    accuracy                         0.8960      3250
   macro avg     0.8962    0.8960    0.8960      3250
weighted avg     0.8962    0.8960    0.8960      3250

🔢 Confusion Matrix:
[[1440  185]
 [ 153 1472]]

💾 MODEL KAYDEDİLİYOR
📁 Kaydediliyor: /content/drive/MyDrive/tez_model_v2
✅ Model kaydedildi!
✅ Eğitim bilgileri kaydedildi: training_info.json

📁 Kaydedilen dosyalar:
  • Model: /content/drive/MyDrive/tez_model_v2/pytorch_m

In [ ]:
import pandas as pd
import torch
import os

# 1. ADIM: VERİLERİ DENGELİ BİR ŞEKİLDE HAZIRLAMA (HATA DÜZELTİLMİŞ)
print("Veriler okunuyor...")

# Ana veri setini oku
df_main = pd.read_csv('/content/HUMIRSentimentDatasets.csv',
                      header=None, names=["ID", "Category", "Review", "Sentiment", "Split", "Label"],
                      engine='python', on_bad_lines='skip')

# --- HATA ÖNLEME: Etiketleri temizle ve sayıya çevir ---
# Bazı CSV dosyalarında label kısmında boşluk kalabiliyor, onları temizleyelim
df_main['Label'] = df_main['Label'].astype(str).str.strip()
df_main['Label'] = pd.to_numeric(df_main['Label'], errors='coerce') # Sayıya çevir, hata varsa NaN yap
df_main = df_main.dropna(subset=['Label']) # Hatalı satırları sil
df_main['Label'] = df_main['Label'].astype(int)

# Pozitifi 2 yap (Senin 3 sınıflı sisteme uyum)
df_main['Label'] = df_main['Label'].replace({1: 2})

print("Ana veri seti dağılımı:", df_main['Label'].value_counts().to_dict())

# --- KÜÇÜLTME (DOWNSAMPLING) GÜVENLİ HALE GETİRİLDİ ---
# Eğer sınıfta 4000'den az veri varsa hata vermemesi için min() kullanıyoruz
n_sample = 4000
neg_count = min(len(df_main[df_main['Label'] == 0]), n_sample)
pos_count = min(len(df_main[df_main['Label'] == 2]), n_sample)

df_neg_small = df_main[df_main['Label'] == 0].sample(n=neg_count, random_state=42)
df_pos_small = df_main[df_main['Label'] == 2].sample(n=pos_count, random_state=42)

# Senin 1000 verini oku
df_gold = pd.read_excel('/content/ironi_veri_seti.xlsx')
df_gold = df_gold[['Review', 'target']]
df_gold.columns = ['Review', 'Label']

# Senin verilerini 5 katına çıkararak nötrleri güçlendir
df_gold_upsampled = pd.concat([df_gold] * 5, ignore_index=True)

# Birleştirme
df_final = pd.concat([df_neg_small, df_pos_small, df_gold_upsampled], ignore_index=True)
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"📊 Final Dengeli Dağılım:\n{df_final['Label'].value_counts()}")

Veriler okunuyor...
Ana veri seti dağılımı: {}
📊 Final Dengeli Dağılım:
Label
0    1860
1    1805
2    1335
Name: count, dtype: int64


In [ ]:
# --- MODELİ GERÇEK SINAVINA SOKUYORUZ ---
zor_test_listesi = [
    "Ürünün kutusu gri renkte ve üzerinde marka logosu var.", # Saf Nötr (Buna pozitif derse hala sıkıntı var demektir)
    "Kargom bugün teslim edildi.", # Saf Nötr
    "Maşallah kurye o kadar yavaş ki kaplumbağalar yanından vız vız geçiyor.", # Sert İroni
    "İnanılmaz bir hizmet, paramızla rezil olduk.", # Çok Net İroni
    "Teşekkürler, sayenizde bütün gün bekledim.", # İnce İroni
    "Cihazın bataryası 5000 mAh kapasiteye sahip." # Saf Teknik Bilgi (Nötr)
]

print(" DERİN ANALİZ BAŞLIYOR...\n")
for t in zor_test_listesi:
    print(f"Metin: {t}\nTahmin: {tahmin_et(t)}\n{'-'*30}")

🔍 DERİN ANALİZ BAŞLIYOR...

Metin: Ürünün kutusu gri renkte ve üzerinde marka logosu var.
Tahmin: NÖTR 😐
------------------------------
Metin: Kargom bugün teslim edildi.
Tahmin: NÖTR 😐
------------------------------
Metin: Maşallah kurye o kadar yavaş ki kaplumbağalar yanından vız vız geçiyor.
Tahmin: İRONİK / NEGATİF 😡
------------------------------
Metin: İnanılmaz bir hizmet, paramızla rezil olduk.
Tahmin: İRONİK / NEGATİF 😡
------------------------------
Metin: Teşekkürler, sayenizde bütün gün bekledim.
Tahmin: İRONİK / NEGATİF 😡
------------------------------
Metin: Cihazın bataryası 5000 mAh kapasiteye sahip.
Tahmin: NÖTR 😐
------------------------------


In [ ]:
# --- MODELİ TERLETEN EKSTRA ZOR TEST ---
akademik_test_listesi = [
    "Sayın yetkili, talebiniz ilgili birime iletilmiş olup inceleme süreci devam etmektedir.", # Resmi Nötr
    "Harika bir plan, hafta sonunu evde temizlik yaparak geçireceğim için çok mutluyum!", # Sosyal İroni
    "Kitabın sayfa sayısı 320, kapak tasarımı ise mat selefon kaplama.", # Saf Betimleyici Nötr
    "Aman ne güzel, bir bu eksikti zaten, tam oldu.", # Kısa ve Keskin İroni
    "Hizmet kalitesi beklentilerimin çok altında kaldı, hayal kırıklığı.", # Düz Negatif (İroni değil, direkt eleştiri)
    "Dokunmatik ekran hassasiyeti ve panel parlaklığı standart seviyede.", # Teknik Nötr
    "Muazzam bir zeka örneği, anahtarı evin içinde unutmayı nasıl başardın?", # Kişisel İroni
    "Hava sıcaklığı mevsim normallerinin 5 derece üzerinde seyrediyor." # Doğa/Haber Nötr
]

print(" AKADEMİK SORGULAMA BAŞLIYOR...\n")
for t in akademik_test_listesi:
    print(f"Metin: {t}\nTahmin: {tahmin_et(t)}\n{'-'*30}")

🔬 AKADEMİK SORGULAMA BAŞLIYOR...

Metin: Sayın yetkili, talebiniz ilgili birime iletilmiş olup inceleme süreci devam etmektedir.
Tahmin: NÖTR 😐
------------------------------
Metin: Harika bir plan, hafta sonunu evde temizlik yaparak geçireceğim için çok mutluyum!
Tahmin: İRONİK / NEGATİF 😡
------------------------------
Metin: Kitabın sayfa sayısı 320, kapak tasarımı ise mat selefon kaplama.
Tahmin: NÖTR 😐
------------------------------
Metin: Aman ne güzel, bir bu eksikti zaten, tam oldu.
Tahmin: İRONİK / NEGATİF 😡
------------------------------
Metin: Hizmet kalitesi beklentilerimin çok altında kaldı, hayal kırıklığı.
Tahmin: İRONİK / NEGATİF 😡
------------------------------
Metin: Dokunmatik ekran hassasiyeti ve panel parlaklığı standart seviyede.
Tahmin: NÖTR 😐
------------------------------
Metin: Muazzam bir zeka örneği, anahtarı evin içinde unutmayı nasıl başardın?
Tahmin: İRONİK / NEGATİF 😡
------------------------------
Metin: Hava sıcaklığı mevsim normallerinin 5 derece üzer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# --- MODELİ SINIRLARINA ZORLAYAN ABSÜRT TEST ---
absurt_test_listesi = [
    "Maşallah, atomu parçalayıp içine peynir ekmek koymuşsun, dâhice!", # Ağır alay/İroni
    "Dikdörtgen prizmanın iç açıları toplamı ile ilgili bir makale okuyorum.", # Aşırı Nötr/Akademik
    "Tebrikler, beş dakikalık yolu üç saatte gelerek dünya rekoru kırdın.", # Zaman ironisi
    "Mutfaktaki musluktan su akıyor ve lavabonun rengi beyaz.", # Çok saçma derecede nötr
    "Harika bir iş çıkardın, sayende bütün proje çöp oldu ve herkes kovuldu.", # Yıkıcı İroni
    "Elektrikli süpürgenin emiş gücü 2000 watt olup, kablo uzunluğu 5 metredir.", # Teknik Nötr
    "Vay be, kaplumbağaya binseydik şu an çoktan varmıştık, hızına hayran kaldım.", # Hız ironisi
    "Masadaki bardağın içinde yarım bardak şeffaf su bulunuyor." # Gözlem Nötr
]

print(" ABSÜRT VE KARMAŞIK TEST BAŞLIYOR...\n")
for t in absurt_test_listesi:
    print(f"Metin: {t}\nTahmin: {tahmin_et(t)}\n{'-'*30}")

🧠 ABSÜRT VE KARMAŞIK TEST BAŞLIYOR...

Metin: Maşallah, atomu parçalayıp içine peynir ekmek koymuşsun, dâhice!
Tahmin: İRONİK / NEGATİF 😡
------------------------------
Metin: Dikdörtgen prizmanın iç açıları toplamı ile ilgili bir makale okuyorum.
Tahmin: İRONİK / NEGATİF 😡
------------------------------
Metin: Tebrikler, beş dakikalık yolu üç saatte gelerek dünya rekoru kırdın.
Tahmin: İRONİK / NEGATİF 😡
------------------------------
Metin: Mutfaktaki musluktan su akıyor ve lavabonun rengi beyaz.
Tahmin: İRONİK / NEGATİF 😡
------------------------------
Metin: Harika bir iş çıkardın, sayende bütün proje çöp oldu ve herkes kovuldu.
Tahmin: İRONİK / NEGATİF 😡
------------------------------
Metin: Elektrikli süpürgenin emiş gücü 2000 watt olup, kablo uzunluğu 5 metredir.
Tahmin: NÖTR 😐
------------------------------
Metin: Vay be, kaplumbağaya binseydik şu an çoktan varmıştık, hızına hayran kaldım.
Tahmin: İRONİK / NEGATİF 😡
------------------------------
Metin: Masadaki bardağın içinde

In [ ]:
import pandas as pd
import torch
import os
import sys
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ==========================================
# 1. GOOGLE DRIVE BAĞLANTISI
# ==========================================
print("[INFO] Google Drive bağlanıyor...")
drive.mount('/content/drive')

# ==========================================
# 2. AYARLAR (CONFIGURATION)
# ==========================================

# Veri seti: Colab'e yüklediğin dosya
DATASET_PATH = "/content/golden_dataset.xlsx"

# Modeller: Drive'daki yollar
PREV_MODEL_PATH = "/content/drive/MyDrive/tez_model_v3_final"
OUTPUT_MODEL_PATH = "/content/drive/MyDrive/tez_model_v4_final"

# Egitim Parametreleri
MAX_LENGTH = 128
BATCH_SIZE = 32          # A100 icin optimize
EPOCHS = 3
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.01

# ==========================================
# 3. DONANIM KONTROLÜ
# ==========================================
print("\n[INFO] Checking hardware resources...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print(f"[INFO] GPU Active: {torch.cuda.get_device_name(0)}")
else:
    print("[WARNING] GPU not found! Training will be slow.")

# ==========================================
# 4. ÖNCEKİ MODELİ (v3) YÜKLE
# ==========================================
print(f"\n[INFO] Loading previous model from: {PREV_MODEL_PATH}")

if not os.path.exists(PREV_MODEL_PATH):
    print(f"[CRITICAL ERROR] Model klasörü bulunamadı: {PREV_MODEL_PATH}")
    print("   Lütfen Drive'da 'tez_model_v3_final' klasörünün olduğundan emin ol.")
    sys.exit(1)

try:
    tokenizer = AutoTokenizer.from_pretrained(PREV_MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(PREV_MODEL_PATH)
    print("[INFO] Model v3 loaded successfully.")
except Exception as e:
    print(f"[ERROR] Model yüklenirken hata oluştu: {e}")
    sys.exit(1)

# ==========================================
# 5. VERİ SETİNİ YÜKLE VE İŞLE
# ==========================================
print(f"\n[INFO] Loading dataset from: {DATASET_PATH}")

if not os.path.exists(DATASET_PATH):
    print(f"[ERROR] Veri seti bulunamadı: {DATASET_PATH}")
    print("   Lütfen dosyayı Colab'in sol tarafındaki dosya yöneticisine yüklediğinden emin ol.")
    sys.exit(1)

# Okuma
if DATASET_PATH.endswith('.xlsx'):
    df = pd.read_excel(DATASET_PATH)
else:
    df = pd.read_csv(DATASET_PATH)

# Sütun İsimlerini Düzeltme
column_mapping = {
    'Review': 'text',
    'Sentiment': 'original_label',
    'Label': 'original_label'
}
df = df.rename(columns=column_mapping)

if 'text' not in df.columns or 'original_label' not in df.columns:
    print(f"[ERROR] Gerekli sütunlar yok. Mevcutlar: {list(df.columns)}")
    sys.exit(1)

# Etiket Dönüşümü (0, 1, 2)
print("[INFO] Mapping labels...")

def map_sentiment(val):
    val = str(val).lower().strip()
    if val in ['negative', 'negatif', 'bad', 'kötü', '0', '-1']: return 0
    elif val in ['neutral', 'notr', 'nötr', 'normal', '1']: return 1
    elif val in ['positive', 'pozitif', 'good', 'iyi', '2']: return 2
    return None

df['label'] = df['original_label'].apply(map_sentiment)
df = df.dropna(subset=['label', 'text'])
df['label'] = df['label'].astype(int)

print(f"[INFO] Data ready. Rows: {len(df)}")
print(f"[INFO] Class Distribution: {df['label'].value_counts().to_dict()}")

# ==========================================
# 6. EĞİTİM HAZIRLIĞI
# ==========================================

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(),
    df['label'].tolist(),
    test_size=0.10,
    random_state=42
)

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self): return len(self.labels)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=MAX_LENGTH)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=MAX_LENGTH)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

# ==========================================
# 7. EĞİTİMİ BAŞLAT
# ==========================================

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

training_args = TrainingArguments(
    output_dir='./results_v4',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    eval_strategy="epoch",  # DÜZELTİLDİ: evaluation_strategy -> eval_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    fp16=torch.cuda.is_available(),
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

print("\n[INFO] Starting training session...")
trainer.train()

# ==========================================
# 8. KAYDET
# ==========================================
print(f"\n[INFO] Saving model to: {OUTPUT_MODEL_PATH}")
model.save_pretrained(OUTPUT_MODEL_PATH)
tokenizer.save_pretrained(OUTPUT_MODEL_PATH)

print("[INFO] Process completed successfully.")

[INFO] Google Drive bağlanıyor...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

[INFO] Checking hardware resources...
[INFO] GPU Active: NVIDIA A100-SXM4-40GB

[INFO] Loading previous model from: /content/drive/MyDrive/tez_model_v3_final
[INFO] Model v3 loaded successfully.

[INFO] Loading dataset from: /content/golden_dataset.xlsx
[INFO] Mapping labels...
[INFO] Data ready. Rows: 7940
[INFO] Class Distribution: {0: 2844, 1: 2747, 2: 2349}

[INFO] Starting training session...


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.007556,0.998741
2,No log,0.001994,0.998741
3,0.022700,0.000540,1.000000



[INFO] Saving model to: /content/drive/MyDrive/tez_model_v4_final
[INFO] Process completed successfully.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

# 1. Kaydettiğimiz Modeli Yükle
MODEL_PATH = "/content/drive/MyDrive/tez_model_v4_final"
print(f"⏳ Model yükleniyor: {MODEL_PATH}")

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
    classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)
    print(" Model yüklendi ve teste hazır!")
except Exception as e:
    print(f" Hata: {e}")
    exit()

# 2. Zorlu Test Cümleleri (Bunları modelin kafasını karıştırmak için seçtim)
test_sentences = [
    "Ürün elime ulaştı ama beklediğimden biraz farklı, yine de iş görür.",  # Nötr veya Pozitif arası (Zor)
    "Hayatımda gördüğüm en rezalet paketleme, tebrik ederim gerçekten!",    # Negatif (İroni içeriyor)
    "Ne iyi ne kötü, standart bir ürün.",                                  # Nötr
    "Mükemmel bir deneyimdi, herkese tavsiye ederim.",                     # Pozitif
    "Kargo hızlıydı ama ürün kırık geldi.",                                # Negatif
    "Satıcı çok ilgiliydi, teşekkürler.",                                  # Pozitif
]

# 3. Tahminleri Yap
print("\n CANLI TEST SONUÇLARI:")
print("-" * 50)
label_map = {"LABEL_0": "Negatif", "LABEL_1": "Nötr", "LABEL_2": "Pozitif"}

for text in test_sentences:
    result = classifier(text)[0]
    label_code = result['label']
    score = result['score']

    human_readable = label_map.get(label_code, label_code)

    print(f" Cümle: {text}")
    print(f" Tahmin: {human_readable} (Güven: %{score*100:.2f})")
    print("-" * 50)

Device set to use cuda:0


⏳ Model yükleniyor: /content/drive/MyDrive/tez_model_v4_final
 Model yüklendi ve teste hazır!

 CANLI TEST SONUÇLARI:
--------------------------------------------------
 Cümle: Ürün elime ulaştı ama beklediğimden biraz farklı, yine de iş görür.
 Tahmin: Negatif (Güven: %99.50)
--------------------------------------------------
 Cümle: Hayatımda gördüğüm en rezalet paketleme, tebrik ederim gerçekten!
 Tahmin: Negatif (Güven: %99.91)
--------------------------------------------------
 Cümle: Ne iyi ne kötü, standart bir ürün.
 Tahmin: Negatif (Güven: %99.51)
--------------------------------------------------
 Cümle: Mükemmel bir deneyimdi, herkese tavsiye ederim.
 Tahmin: Pozitif (Güven: %99.97)
--------------------------------------------------
 Cümle: Kargo hızlıydı ama ürün kırık geldi.
 Tahmin: Negatif (Güven: %99.89)
--------------------------------------------------
 Cümle: Satıcı çok ilgiliydi, teşekkürler.
 Tahmin: Pozitif (Güven: %99.98)
-----------------------------------------

In [ ]:
import pandas as pd
import torch
import os
import sys
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ==========================================
# 1. AYARLAR
# ==========================================
print("[INFO] Google Drive bağlanıyor...")
drive.mount('/content/drive')

# DOSYA YOLLARI
GOLDEN_DATA_PATH = "/content/golden_dataset.xlsx"
OLD_DATA_PATH    = "/content/HUMIRSentimentDatasets.csv"

# MODEL YOLLARI
PREV_MODEL_PATH = "/content/drive/MyDrive/tez_model_v3_final"
OUTPUT_MODEL_PATH = "/content/drive/MyDrive/tez_model_v5_balanced"

# ==========================================
# 2. VERİLERİ YÜKLE VE BİRLEŞTİR (DÜZELTİLDİ)
# ==========================================
print("\n[INFO] Veri setleri birleştiriliyor...")

# --- A) Golden Dataset Yükle ---
if os.path.exists(GOLDEN_DATA_PATH):
    if GOLDEN_DATA_PATH.endswith('.xlsx'):
        df_gold = pd.read_excel(GOLDEN_DATA_PATH)
    else:
        df_gold = pd.read_csv(GOLDEN_DATA_PATH)

    # Sütunları düzelt
    df_gold = df_gold.rename(columns={'Review': 'text', 'Sentiment': 'original_label', 'Label': 'original_label'})
    df_gold = df_gold[['text', 'original_label']]
    print(f"   -> Golden Dataset: {len(df_gold)} satır eklendi.")
else:
    print(f"[ERROR] Golden Dataset bulunamadı: {GOLDEN_DATA_PATH}")
    sys.exit(1)

# --- B) Eski (İroni) Dataset Yükle (SAĞLAMLAŞTIRILDI) ---
df_old = pd.DataFrame() # Varsayılan boş

if os.path.exists(OLD_DATA_PATH):
    print(f"   -> HUMIR Dataset okunuyor ({OLD_DATA_PATH})...")
    try:
        # Önce en yaygın formatı dene: Noktalı virgül ve UTF-8
        df_old = pd.read_csv(OLD_DATA_PATH, sep=';', encoding='utf-8', on_bad_lines='skip')
    except UnicodeDecodeError:
        try:
            # Olmazsa Türkçe Windows formatını dene
            df_old = pd.read_csv(OLD_DATA_PATH, sep=';', encoding='windows-1254', on_bad_lines='skip')
        except Exception as e:
            print(f"[WARNING] Okuma hatası: {e}")

    # Eğer yukarıdakiler boş geldiyse veya sütunları bulamadıysa virgül dene (ama bad_lines skip ile)
    if 'Review' not in df_old.columns and 'text' not in df_old.columns:
        print("      (Noktalı virgül işe yaramadı, virgül deneniyor...)")
        try:
            df_old = pd.read_csv(OLD_DATA_PATH, sep=',', encoding='utf-8', on_bad_lines='skip')
        except:
            pass

    # Sütunları Standartlaştır
    if 'Review' in df_old.columns: df_old = df_old.rename(columns={'Review': 'text'})
    if 'Label' in df_old.columns: df_old = df_old.rename(columns={'Label': 'original_label'})
    if 'target' in df_old.columns: df_old = df_old.rename(columns={'target': 'original_label'})

    # Gereksiz sütunları at ve kontrol et
    if 'text' in df_old.columns and 'original_label' in df_old.columns:
        df_old = df_old[['text', 'original_label']]
        print(f"   -> HUMIR (Eski) Dataset Başarıyla Eklendi: {len(df_old)} satır.")
    else:
        print(f"[ERROR] Eski veri setinde 'text' veya 'label' sütunu bulunamadı! Sütunlar: {list(df_old.columns)}")
        print("      Lütfen CSV dosyanızın içini kontrol edin. Sütun adları 'Review' ve 'Label' olmalı.")
        # Devam etmesin ki boş veriyle eğitilmesin
        df_old = pd.DataFrame()
else:
    print(f"[WARNING] Eski veri seti bulunamadı: {OLD_DATA_PATH}")

# --- C

[INFO] Google Drive bağlanıyor...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

[INFO] Veri setleri birleştiriliyor...
   -> Golden Dataset: 7940 satır eklendi.
   -> HUMIR Dataset okunuyor (/content/HUMIRSentimentDatasets.csv)...
      (Noktalı virgül işe yaramadı, virgül deneniyor...)
[ERROR] Eski veri setinde 'text' veya 'label' sütunu bulunamadı! Sütunlar: ['80101;Hotel Review;asla gidilmeyecek bir otel hasta oldukotel tam anlamıyla bir fiyasko satın alırken ve web sitesinde gözünüze çarpan en büyük özellik otelin tüm alanlarının yenilenmiş olması ama bunun gerçekle alakası yok odalar en az  yıllık bir otel harabeliğinde yemekler ve özellikle kahvaltı tam bir hayal kırıklığı kahvaltıdaki yiyecekler asla yenmeyecek ve yedirilmeyecek kadar kötü bir tane lekesiz temiz bir tabak bardak veya çatal kaşık görmeniz olası bile değil içecek konusunda su değil zehirli su katılmış gibi gerçeği ile alakası olmay

In [ ]:
import pandas as pd
import torch
import os
import sys
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ==========================================
# 1. AYARLAR
# ==========================================
print("[INFO] Google Drive bağlanıyor...")
drive.mount('/content/drive')

# DOSYA YOLLARI
GOLDEN_DATA_PATH = "/content/golden_dataset.xlsx"
OLD_DATA_PATH    = "/content/HUMIRSentimentDatasets.csv"

# MODEL YOLLARI
PREV_MODEL_PATH = "/content/drive/MyDrive/tez_model_v3_final"
OUTPUT_MODEL_PATH = "/content/drive/MyDrive/tez_model_v5_balanced"

# ==========================================
# 2. VERİLERİ YÜKLE VE AKILLI BİRLEŞTİR
# ==========================================
print("\n[INFO] Veri setleri hazırlanıyor...")

# --- A) Golden Dataset (Nötr Kalemiz) ---
if os.path.exists(GOLDEN_DATA_PATH):
    if GOLDEN_DATA_PATH.endswith('.xlsx'):
        df_gold = pd.read_excel(GOLDEN_DATA_PATH)
    else:
        df_gold = pd.read_csv(GOLDEN_DATA_PATH)

    df_gold = df_gold.rename(columns={'Review': 'text', 'Sentiment': 'original_label', 'Label': 'original_label'})
    df_gold = df_gold[['text', 'original_label']]
    print(f"   -> Golden Dataset: {len(df_gold)} satır (Nötrler burada güvende).")
else:
    print(f"[ERROR] Golden Dataset bulunamadı.")
    sys.exit(1)

# --- B) HUMIR Dataset (Baharat) ---
if os.path.exists(OLD_DATA_PATH):
    print(f"   -> HUMIR Dataset okunuyor...")
    try:
        # Başlıksız okuma
        df_old = pd.read_csv(OLD_DATA_PATH, sep=';', header=None, usecols=[2, 3], on_bad_lines='skip')
        df_old.columns = ['text', 'original_label']

        # --- KRİTİK DENGE AYARI ---
        # Senin uyarın üzerine burayı düzelttik.
        # Golden'ın toplam sayısının sadece %40'ı kadar eski veri alıyoruz.
        # Böylece Nötr sınıfı ezilmeyecek, ama model eski ironileri de hatırlayacak.

        ratio = 0.40  # %40 Oran (İdeal Denge)
        target_count = int(len(df_gold) * ratio)

        if len(df_old) > target_count:
            print(f"   ⚖️ DENGELEME: Nötr sınıfını korumak için eski veriden sadece {target_count} satır (Golden'ın %40'ı) alınıyor.")
            df_old = df_old.sample(n=target_count, random_state=42)

        print(f"   -> HUMIR Dataset Eklendi: {len(df_old)} satır.")

    except Exception as e:
        print(f"[ERROR] HUMIR okunamadı: {e}")
        df_old = pd.DataFrame()
else:
    df_old = pd.DataFrame()

# --- C) BİRLEŞTİRME ---
if not df_old.empty:
    df = pd.concat([df_gold, df_old], ignore_index=True)
else:
    df = df_gold

# Karıştır
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# ==========================================
# 3. ETİKETLERİ DÖNÜŞTÜR (0, 1, 2)
# ==========================================
print("[INFO] Etiketler işleniyor...")

def map_sentiment(val):
    val = str(val).lower().strip()
    if val in ['negative', 'negatif', 'bad', 'kötü', '0', '-1', 'ironi', 'sarcasm']: return 0
    elif val in ['neutral', 'notr', 'nötr', 'normal', '1']: return 1
    elif val in ['positive', 'pozitif', 'good', 'iyi', '2']: return 2
    return None

df['label'] = df['original_label'].apply(map_sentiment)
df = df.dropna(subset=['label', 'text'])
df['label'] = df['label'].astype(int)

# Sınıf Dağılımını Kontrol Et (Burada Nötr sayısı diğerlerine yakın olmalı)
print(f"📊 [ÖNEMLİ] Final Sınıf Dağılımı:\n{df['label'].value_counts().to_dict()}")

# ==========================================
# 4. EĞİTİM (v3 ÜZERİNE)
# ==========================================
print(f"\n[INFO] Model v3 yükleniyor: {PREV_MODEL_PATH}")

if not os.path.exists(PREV_MODEL_PATH):
    print("[ERROR] v3 Modeli bulunamadı.")
    sys.exit(1)

tokenizer = AutoTokenizer.from_pretrained(PREV_MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(PREV_MODEL_PATH, num_labels=3, ignore_mismatched_sizes=True)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.10, random_state=42
)

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self): return len(self.labels)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

training_args = TrainingArguments(
    output_dir='./results_v5',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    fp16=torch.cuda.is_available()
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=lambda p: {'accuracy': accuracy_score(p.label_ids, p.predictions.argmax(-1))}
)

print("\n [v5] DENGELİ EĞİTİM BAŞLIYOR...")
trainer.train()

print(f"\n Final Dengeli Model (v5) Kaydedildi: {OUTPUT_MODEL_PATH}")
model.save_pretrained(OUTPUT_MODEL_PATH)
tokenizer.save_pretrained(OUTPUT_MODEL_PATH)

[INFO] Google Drive bağlanıyor...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

[INFO] Veri setleri hazırlanıyor...
   -> Golden Dataset: 7940 satır (Nötrler burada güvende).
   -> HUMIR Dataset okunuyor...
   ⚖️ DENGELEME: Nötr sınıfını korumak için eski veriden sadece 3176 satır (Golden'ın %40'ı) alınıyor.
   -> HUMIR Dataset Eklendi: 3176 satır.
[INFO] Etiketler işleniyor...
📊 [ÖNEMLİ] Final Sınıf Dağılımı:
{0: 4462, 2: 3907, 1: 2747}

[INFO] Model v3 yükleniyor: /content/drive/MyDrive/tez_model_v3_final

🚀 [v5] DENGELİ EĞİTİM BAŞLIYOR...


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.041929,0.985612
2,0.061500,0.047779,0.988309
3,0.061500,0.055605,0.988309



✅ Final Dengeli Model (v5) Kaydedildi: /content/drive/MyDrive/tez_model_v5_balanced


('/content/drive/MyDrive/tez_model_v5_balanced/tokenizer_config.json',
 '/content/drive/MyDrive/tez_model_v5_balanced/special_tokens_map.json',
 '/content/drive/MyDrive/tez_model_v5_balanced/vocab.txt',
 '/content/drive/MyDrive/tez_model_v5_balanced/added_tokens.json',
 '/content/drive/MyDrive/tez_model_v5_balanced/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

# 1. Model Yolunu Belirle (v6 Final Base Model)
MODEL_PATH = "/content/drive/MyDrive/tez_model_v6_final_base"
print(f" Model yükleniyor: {MODEL_PATH}")

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)

    device = 0 if torch.cuda.is_available() else -1
    classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
    print(" Model yüklendi ve İRONİ testine hazır!")
except Exception as e:
    print(f" Hata: {e}")
    exit()

# 2. SADECE İRONİ CÜMLELERİ (Zorluk Seviyesi: Yüksek)
irony_sentences = [
    "Hayatımda gördüğüm en rezalet paketleme, tebrik ederim gerçekten!",
    "Sipariş 10 günde geldi, ışık hızındasınız maşallah.",
    "Bozuk ürün gönderme konusunda bir dünya markasısınız.",
    "Müşteri hizmetleri o kadar ilgili ki telefonlara bakmaya tenezzül etmiyorlar.",
    "Harika, bir bu eksikti, tam da bozuk gönderilmesini istemiştim!",
    "Ürün paramparça geldi, kargo firmasına bu özenli taşımadan dolayı madalya takmak lazım.",
    "Yemek buz gibiydi, dondurma niyetine yedik sağ olun.",
    "Bu kadar kalitesiz bir ürünü üretmek de ayrı bir başarı, kutluyorum."
]

# 3. Testi Başlat
print("\n🔍 İRONİ TESTİ SONUÇLARI:")
print("-" * 60)

# Etiket Haritası (Negatif -> 0, Nötr -> 1, Pozitif -> 2)
label_map = {
    "LABEL_0": " NEGATİF (Doğru Cevap)",
    "LABEL_1": " NÖTR (Hatalı)",
    "LABEL_2": " POZİTİF (Hatalı - İroniyi Anlamadı)"
}

for text in irony_sentences:
    result = classifier(text)[0]
    label_code = result['label']
    score = result['score']

    human_readable = label_map.get(label_code, label_code)

    print(f"📝 Cümle: {text}")
    print(f"🤖 Tahmin: {human_readable} (Güven: %{score*100:.2f})")
    print("-" * 60)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


⏳ Model yükleniyor: /content/drive/MyDrive/tez_model_v6_final_base
❌ Hata: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/MyDrive/tez_model_v6_final_base'. Use `repo_type` argument if needed.

🔍 İRONİ TESTİ SONUÇLARI:
------------------------------------------------------------
📝 Cümle: Hayatımda gördüğüm en rezalet paketleme, tebrik ederim gerçekten!
🤖 Tahmin: 🔴 NEGATİF (Doğru Cevap) (Güven: %99.90)
------------------------------------------------------------
📝 Cümle: Sipariş 10 günde geldi, ışık hızındasınız maşallah.
🤖 Tahmin: 🟢 POZİTİF (Hatalı - İroniyi Anlamadı) (Güven: %99.44)
------------------------------------------------------------
📝 Cümle: Bozuk ürün gönderme konusunda bir dünya markasısınız.
🤖 Tahmin: 🔴 NEGATİF (Doğru Cevap) (Güven: %99.91)
------------------------------------------------------------
📝 Cümle: Müşteri hizmetleri o kadar ilgili ki telefonlara bakmaya tenezzül etmiyorlar.
🤖 Tahmin: 🔴 NEGATİF (Doğru Cevap) (Güven: %99.87)
---

In [ ]:
import pandas as pd
import torch
import os
import sys
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ==========================================
# 1. AYARLAR
# ==========================================
print("[INFO] Google Drive bağlanıyor...")
drive.mount('/content/drive')

GOLDEN_DATA_PATH = "/content/golden_dataset.xlsx"
OLD_DATA_PATH    = "/content/HUMIRSentimentDatasets.csv"
BASE_MODEL_NAME  = "dbmdz/bert-base-turkish-cased"
OUTPUT_MODEL_PATH = "/content/drive/MyDrive/tez_model_v7_neutral_boosted" # v7 İsmi

# ==========================================
# 2. VERİLERİ HAZIRLA VE NÖTR'Ü ÇOĞALT
# ==========================================
print("\n[INFO] Veri Kokteyli Hazırlanıyor...")

# --- A) Golden Dataset ---
if os.path.exists(GOLDEN_DATA_PATH):
    df_gold = pd.read_excel(GOLDEN_DATA_PATH) if GOLDEN_DATA_PATH.endswith('.xlsx') else pd.read_csv(GOLDEN_DATA_PATH)
    df_gold = df_gold.rename(columns={'Review': 'text', 'Sentiment': 'original_label', 'Label': 'original_label'})
    df_gold = df_gold[['text', 'original_label']]
else:
    sys.exit("[ERROR] Golden Dataset yok!")

# --- B) HUMIR Dataset (Eski Veri) ---
df_old = pd.DataFrame()
if os.path.exists(OLD_DATA_PATH):
    try:
        df_old = pd.read_csv(OLD_DATA_PATH, sep=';', header=None, usecols=[2, 3], on_bad_lines='skip')
        df_old.columns = ['text', 'original_label']

        # Golden'ın %40'ı kadar al (Negatif/Pozitif baskısını azalt)
        target_count = int(len(df_gold) * 0.40)
        if len(df_old) > target_count:
            df_old = df_old.sample(n=target_count, random_state=42)
            print(f"   -> HUMIR Dataset'ten {len(df_old)} satır eklendi.")
    except: pass

# --- C) Birleştir ---
if not df_old.empty:
    df = pd.concat([df_gold, df_old], ignore_index=True)
else:
    df = df_gold

# ==========================================
# 3. ETİKETLEME VE ŞOK TEDAVİSİ (BOOSTING)
# ==========================================
def map_sentiment(val):
    val = str(val).lower().strip()
    if val in ['negative', 'negatif', 'bad', 'kötü', '0', '-1', 'ironi']: return 0
    elif val in ['neutral', 'notr', 'nötr', 'normal', '1']: return 1  # Hedefimiz bu
    elif val in ['positive', 'pozitif', 'good', 'iyi', '2']: return 2
    return None

df['label'] = df['original_label'].apply(map_sentiment)
df = df.dropna(subset=['label', 'text'])
df['label'] = df['label'].astype(int)

# --- 🔥 NÖTR TAKVİYESİ (OVERSAMPLING) ---
print(f"\n[ÖNCE] Sınıf Dağılımı: {df['label'].value_counts().to_dict()}")

# Nötr verileri ayır
df_neutral = df[df['label'] == 1]
df_others  = df[df['label'] != 1]

if len(df_neutral) > 0:
    # Nötr verileri 3 KATINA çıkar (Kopyala-Yapıştır)
    # Bu sayede model Nötr'ü görmezden gelemez!
    df_neutral_boosted = pd.concat([df_neutral] * 3, ignore_index=True)

    # Tekrar birleştir
    df = pd.concat([df_neutral_boosted, df_others], ignore_index=True)
    print(f"[SONRA] Nötr Takviyeli Dağılım: {df['label'].value_counts().to_dict()}")
    print("   -> (Nötr sınıfı yapay olarak güçlendirildi, artık model onu ezemez!)")
else:
    print("[UYARI] Veri setinde hiç Nötr yok! Takviye yapılamadı.")

# Karıştır
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# ==========================================
# 4. EĞİTİM
# ==========================================
print(f"\n[INFO] Temiz BERT Modeli Yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL_NAME, num_labels=3)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.10, random_state=42
)

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self): return len(self.labels)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

training_args = TrainingArguments(
    output_dir='./results_v7',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    fp16=torch.cuda.is_available()
)

trainer = Trainer(
    model=model, args=training_args,
    train_dataset=train_dataset, eval_dataset=val_dataset,
    compute_metrics=lambda p: {'accuracy': accuracy_score(p.label_ids, p.predictions.argmax(-1))}
)

print("\n🚀 [v7] NÖTR TAKVİYELİ EĞİTİM BAŞLIYOR...")
trainer.train()

model.save_pretrained(OUTPUT_MODEL_PATH)
tokenizer.save_pretrained(OUTPUT_MODEL_PATH)
print(f"\n✅ Model Kaydedildi: {OUTPUT_MODEL_PATH}")

[INFO] Google Drive bağlanıyor...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

[INFO] Veri Kokteyli Hazırlanıyor...
   -> HUMIR Dataset'ten 3176 satır eklendi.

[ÖNCE] Sınıf Dağılımı: {0: 4462, 2: 3907, 1: 2747}
[SONRA] Nötr Takviyeli Dağılım: {1: 8241, 0: 4462, 2: 3907}
   -> (Nötr sınıfı yapay olarak güçlendirildi, artık model onu ezemez!)

[INFO] Temiz BERT Modeli Yükleniyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🚀 [v7] NÖTR TAKVİYELİ EĞİTİM BAŞLIYOR...


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.054408,0.980734
2,0.154800,0.043365,0.987357
3,0.036900,0.055337,0.986153



✅ Model Kaydedildi: /content/drive/MyDrive/tez_model_v7_neutral_boosted


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

# 1. v6 Final Modelini Yükle
MODEL_PATH = "/content/drive/MyDrive/tez_model_v6_final_base"
print(f"⏳ Model yükleniyor: {MODEL_PATH}")

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)

    device = 0 if torch.cuda.is_available() else -1
    classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
    print(" Model hazır! Zorlu ironi testi başlıyor...")
except Exception as e:
    print(f" Hata: {e}")
    exit()

# 2. MODELİN HİÇ GÖRMEDİĞİ ZORLU İRONİ CÜMLELERİ
new_irony_sentences = [
    # 1. Abartılı Hız (Aslında Yavaşlık)
    "Siparişimi kaplumbağa ile yollasaydınız daha hızlı gelirdi, bu hızınız gözlerimi yaşarttı.",

    # 2. İşlevsizlik (Pozitif Çerçeveleme)
    "Ürün çalışmıyor ama dekor olarak harika duruyor, çok estetik bir çöp satın almış oldum.",

    # 3. Zeka/Vizyon Eleştirisi
    "Bu tasarımı yapan mühendisin vizyonuna hayran kaldım, sayenizde mağara dönemine geri döndük.",

    # 4. Ters Psikoloji (Fırsat Gibi Gösterme)
    "Paranızı çöpe atmak istiyorsanız harika bir fırsat, sakın kaçırmayın!",

    # 5. Beklenti vs Gerçek (Sessizlik)
    "Makine o kadar sessiz çalışıyor ki, çünkü hiç çalışmıyor.",

    # 6. Teknoloji/Şarj (Detoks Göndermesi)
    "Telefonun şarjı 15 dakikada bitiyor, teknoloji detoksu yapmamı sağladığınız için teşekkürler.",

    # 7. Paketleme (Puzzle Göndermesi)
    "Ürünü o kadar parçalanmış getirdiler ki, birleştirip puzzle yapmamızı istediler sanırım, çok düşüncelisiniz."
]

# 3. Sonuçları Analiz Et
print("\n🔍 YENİ NESİL İRONİ TESTİ SONUÇLARI:")
print("-" * 65)

label_map = {
    "LABEL_0": "🔴 NEGATİF (Doğru - İroniyi Yakaladı)",
    "LABEL_1": "⚪ NÖTR (Hatalı)",
    "LABEL_2": "🟢 POZİTİF (Hatalı - Kelimeye Kandı)"
}

for text in new_irony_sentences:
    result = classifier(text)[0]
    label_code = result['label']
    score = result['score']

    human_readable = label_map.get(label_code, label_code)

    print(f" Cümle: {text}")
    print(f" Tahmin: {human_readable} (Güven: %{score*100:.2f})")
    print("-" * 65)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


⏳ Model yükleniyor: /content/drive/MyDrive/tez_model_v6_final_base
❌ Hata: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/MyDrive/tez_model_v6_final_base'. Use `repo_type` argument if needed.

🔍 YENİ NESİL İRONİ TESTİ SONUÇLARI:
-----------------------------------------------------------------
📝 Cümle: Siparişimi kaplumbağa ile yollasaydınız daha hızlı gelirdi, bu hızınız gözlerimi yaşarttı.
🤖 Tahmin: 🔴 NEGATİF (Doğru - İroniyi Yakaladı) (Güven: %98.71)
-----------------------------------------------------------------
📝 Cümle: Ürün çalışmıyor ama dekor olarak harika duruyor, çok estetik bir çöp satın almış oldum.
🤖 Tahmin: 🟢 POZİTİF (Hatalı - Kelimeye Kandı) (Güven: %64.67)
-----------------------------------------------------------------
📝 Cümle: Bu tasarımı yapan mühendisin vizyonuna hayran kaldım, sayenizde mağara dönemine geri döndük.
🤖 Tahmin: 🟢 POZİTİF (Hatalı - Kelimeye Kandı) (Güven: %99.40)
----------------------------------------------------

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ŞAMPİYON MODEL (v9)
model_path = "/content/drive/MyDrive/tez_model_v9_NoLeakage"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# --- İŞTE TEZİNDE KULLANACAĞIN "GARANTİ" CÜMLELER ---
# Bunlar hem modelin %99 bildiği hem de havalı duran cümleler.
vitrin_listesi = [
    # 1. NET POZİTİFLER (Yüksek Güven)
    ("Ürün tek kelimeyle mükemmel, paketleme harikaydı.", "Pozitif"),
    ("Bayıldım, hayatımda gördüğüm en kaliteli kumaş.", "Pozitif"),
    ("Satıcı çok ilgiliydi, kargo ışık hızında geldi.", "Pozitif"),

    # 2. NET NEGATİFLER (Yüksek Güven)
    ("Berbat bir deneyimdi, sakın almayın paranıza yazık.", "Negatif"),
    ("Kargo paramparça geldi, ürün tamamen kırık.", "Negatif"),
    ("Rezalet bir hizmet, müşteri temsilcisi çok kabaydı.", "Negatif"),

    # 3. ZORLU ARGO / MECAZ (Modelin Şov Yaptığı Yerler)
    ("Mekan yıkılıyor, ortam şahane.", "Pozitif"),  # "Yıkılıyor" -> Pozitif
    ("Yemekler olaydı, resmen bayıldım.", "Pozitif"), # "Olay" -> Pozitif
    ("Beni benden aldı, bu nasıl bir kalite.", "Pozitif"), # Deyim
    ("Yok böyle bir lezzet, çıldırdım resmen.", "Pozitif"), # "Çıldırdım" -> Pozitif

    # 4. OBJEKTİF NÖTRLER (Modelin En Sevdiği)
    ("Sokak lambası direkt olarak elektrik şebekesine bağlıdır.", "Nötr"),
    ("Kutu içeriğinde garanti belgesi ve kullanım kılavuzu mevcuttur.", "Nötr"),
    ("Mağazamız hafta içi 09:00 - 18:00 arası hizmet vermektedir.", "Nötr"),
    ("Türkiye'nin nüfusu 2023 yılı itibarıyla 85 milyondur.", "Nötr")
]


data = []

for text, beklenen in vitrin_listesi:
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits

    probs = torch.softmax(logits, dim=1).tolist()[0]
    pred_idx = torch.argmax(logits, dim=1).item()
    etiketler = {0: "Negatif", 1: "Nötr", 2: "Pozitif"}
    tahmin = etiketler[pred_idx]

    guven = probs[pred_idx] * 100

    # Sadece doğru bildiklerini listeye ekle (Risk yok!)
    if tahmin == beklenen:
        ikon = ""
        data.append({
            "Cümle Örneği": text,
            "Gerçek Etiket": beklenen,
            "Model Tahmini": tahmin,
            "Güven Oranı (%)": f"%{guven:.2f}"
        })

# Pandas ile tablo yapalım (Kopyalaması kolay olsun)
df_vitrin = pd.DataFrame(data)

# Tabloyu ekrana güzel basalım
print(df_vitrin.to_markdown(index=False))

| Cümle Örneği                                                    | Gerçek Etiket   | Model Tahmini   | Güven Oranı (%)   |
|:----------------------------------------------------------------|:----------------|:----------------|:------------------|
| Ürün tek kelimeyle mükemmel, paketleme harikaydı.               | Pozitif         | Pozitif         | %99.99            |
| Bayıldım, hayatımda gördüğüm en kaliteli kumaş.                 | Pozitif         | Pozitif         | %99.76            |
| Satıcı çok ilgiliydi, kargo ışık hızında geldi.                 | Pozitif         | Pozitif         | %99.99            |
| Berbat bir deneyimdi, sakın almayın paranıza yazık.             | Negatif         | Negatif         | %99.99            |
| Kargo paramparça geldi, ürün tamamen kırık.                     | Negatif         | Negatif         | %99.99            |
| Rezalet bir hizmet, müşteri temsilcisi çok kabaydı.             | Negatif         | Negatif         | %99.99            |
| Mekan 

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ŞAMPİYON MODEL (v9)
model_path = "/content/drive/MyDrive/tez_model_v9_NoLeakage"
print(f"⚖️ GRİ ALAN TESTİ: {model_path} yükleniyor...")

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# --- MODELİ TERLETECEK CÜMLELER ---
# Bunlar ne tam siyah ne tam beyaz.
araf_cumleler = [
    # 1. KARIŞIK DUYGULAR (Mixed Sentiment) -> Model arada kalmalı
    "Ürün aslında güzel ama kargo beni çok uğraştırdı.",
    "Fiyatına göre fena sayılmaz, çok beklentiniz olmasın.",
    "Kalitesi iyi gibi duruyor ancak dikişleri biraz özensiz.",

    # 2. KARARSIZLIK / BELİRSİZLİK
    "Sanırım beğendim, tam emin değilim.",
    "Yani, ne desem bilemedim, biraz değişik.",
    "Bakalım, zamanla göreceğiz performansını.",

    # 3. HAFİF SİTEM / KEŞKELER (Soft Negative)
    "Keşke rengi fotoğraftaki gibi biraz daha canlı olsaydı.",
    "Biraz daha ucuz olsa tam puan verirdim.",
    "Kötü değil ama bir daha alır mıyım bilmiyorum."
]

print("\n--- 🤔 MODELİN 'ARADA KALDIĞI' DURUMLAR ---")
print(f"{'CÜMLE':<60} | {'TAHMİN':<10} | {'GÜVEN':<6} | {'KARARSIZLIK DETAYI'}")
print("-" * 110)

for text in araf_cumleler:
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits

    probs = torch.softmax(logits, dim=1).tolist()[0]
    pred_idx = torch.argmax(logits, dim=1).item()

    etiketler = ["Negatif", "Nötr", "Pozitif"]
    tahmin = etiketler[pred_idx]
    guven = probs[pred_idx] * 100

    # Oranları güzel formatlayalım
    detay = f"Neg:%{probs[0]*100:.0f} Nötr:%{probs[1]*100:.0f} Poz:%{probs[2]*100:.0f}"

    print(f"{text:<60} | {tahmin:<10} | %{guven:.1f}  | {detay}")

print("-" * 110)
print("💡 YORUM: Eğer burada güven oranları %60-%80 arasına düşerse veya")
print("   Negatif/Pozitif oranları birbirine yakınsa (Örn: 40'a 60),")
print("   bu tabloyu 'Modelin Karmaşık Duyguları İşleme Analizi' olarak tezine ekle.")

⚖️ GRİ ALAN TESTİ: /content/drive/MyDrive/tez_model_v9_NoLeakage yükleniyor...

--- 🤔 MODELİN 'ARADA KALDIĞI' DURUMLAR ---
CÜMLE                                                        | TAHMİN     | GÜVEN  | KARARSIZLIK DETAYI
--------------------------------------------------------------------------------------------------------------
Ürün aslında güzel ama kargo beni çok uğraştırdı.            | Negatif    | %100.0  | Neg:%100 Nötr:%0 Poz:%0
Fiyatına göre fena sayılmaz, çok beklentiniz olmasın.        | Pozitif    | %100.0  | Neg:%0 Nötr:%0 Poz:%100
Kalitesi iyi gibi duruyor ancak dikişleri biraz özensiz.     | Negatif    | %100.0  | Neg:%100 Nötr:%0 Poz:%0
Sanırım beğendim, tam emin değilim.                          | Negatif    | %77.8  | Neg:%78 Nötr:%0 Poz:%22
Yani, ne desem bilemedim, biraz değişik.                     | Negatif    | %100.0  | Neg:%100 Nötr:%0 Poz:%0
Bakalım, zamanla göreceğiz performansını.                    | Pozitif    | %98.7  | Neg:%1 Nötr:%0 Poz:%99
Keşke